In [127]:
# 회귀 문제 정의
# 10개 아울렛 매장에서 1500여개 제품에 대한 판매 데이터를 수집했다. 예측 모델을 만들고 아울렛 특정 매장에서 각 제품의 판매 금액을 예측

In [128]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [129]:
# EDA

In [130]:

# 데이터 크기 확인
print(train.shape, test.shape)
# 데이터 샘플확인
display(train.head(3))
display(test.head(3))
# Item_Outlet_Sales 이 타겟(Y)
# 데이터를 봤을 때 회귀로 풀어야 함

(6818, 12) (1705, 11)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.5,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.6,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.8,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDO38,17.250,Low Fat,0.073251,Canned,76.4986,OUT017,2007,NaN,Tier 2,Supermarket Type1
1,FDO32,6.360,Low Fat,0.120544,Fruits and Vegetables,45.8060,OUT046,1997,Small,Tier 1,Supermarket Type1
2,FDT14,10.695,Regular,0.128247,Dairy,121.8440,OUT018,2009,Medium,Tier 3,Supermarket Type2


In [131]:
# 데이터 타입 확인
train.dtypes
# 7개의 object 타입은 인코딩을 해야함

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [132]:
# 숫자형 데이터의 분포 확인
display(train.describe())

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,5656.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,12.872703,0.066121,140.419533,1997.885890,2190.941459
std,4.651034,0.051383,62.067861,8.339795,1706.131256
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026914,93.610050,1987.000000,836.577700
50%,12.600000,0.053799,142.448300,1999.000000,1806.648300
75%,17.000000,0.095273,185.060150,2004.000000,3115.944000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [133]:
# 문자형 데이터의 분포 확인
display(train.describe(include='O'))
# Item_Identifier의 종류가 1554개로 많아, 원핫인코딩 대신 레이블인코딩 사용이 좋아보임

# test에 있는 데이터도 확인
test.describe(include='O')
# Item_Identifier 를 제외하고 train과 종류가 똑같음

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDW26,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,FDG33,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


In [134]:
# 결측치 확인
print(train.isna().sum())
print('-'*50)
print(test.isna().sum())

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64
--------------------------------------------------
Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64


In [135]:
# 데이터 전처리

In [136]:
# 결측치 처리

# Item_Weight -> 최소값으로
# Outlet_Size -> 최빈값으로

# train
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])
# test
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])

In [137]:
# 인코딩

# 인코딩 대상 컬럼명만 구하기
col_list = list(train.select_dtypes('O').columns)
# Item_Identifier 는 종류가 너무 많으니 제외하기
col_list.pop(0)

print(col_list)

['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [138]:
# train 에서 Y만 분리(Item_Outlet_Sales)
target = train.pop('Item_Outlet_Sales')
print(train.shape, test.shape)

# 인코딩을 위해 train + test concat 하기
df = pd.concat([train,test])
print(df.shape)

(6818, 11) (1705, 11)
(8523, 11)


In [139]:
# sklearn labelencoder 사용하기
# 한 개의 컬럼마다 돌아가면서 해야함
from sklearn.preprocessing import LabelEncoder

for colname in col_list:
    le = LabelEncoder()
    df[colname] = le.fit_transform(df[colname])
    
# 데이터 확인
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,FDW11,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,FDH32,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,FDL52,6.635,2,0.046351,5,37.4506,2,2007,1,1,1
4,FDO09,13.500,2,0.125170,13,261.4910,1,1987,0,2,1


In [142]:
# df를 train, test로 다시 분리

train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()

print(train.shape, test.shape)

(6818, 11) (1705, 11)


In [143]:
# 스케일링, 이상치 처리는 필수 요소가 아니므로 이번에는 제외

In [145]:
# Item_Identifier 종류가 너무 많으므로 이번에는 삭제
print(train.shape, test.shape)
train = train.drop(columns='Item_Identifier')
test = test.drop(columns='Item_Identifier')
print(train.shape, test.shape)

(6818, 11) (1705, 11)
(6818, 10) (1705, 10)


In [146]:
# 검증 데이터 나누기
# train, target(y)-> train, val
# test 유지

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2024)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(5454, 10) (5454,)
(1364, 10) (1364,)


In [147]:
# 머신러닝 학습 및 평가

In [149]:
# RMSE, MSE, MAE, R2 평가 지표로 사용해보기
# RMSE, MSE, MAE 낮을 수록 좋고
# R2 1에 가까울 수록 좋다
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [150]:
# 여러 모델 사용해서 지표가 가장 좋은 것으로 제출하기

In [ ]:
# 모델1 | 선형 회귀, LinearRegression
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()

lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_val)

print(rmse(y_val, y_pred))
print(mean_squared_error(y_val, y_pred))
print(mean_absolute_error(y_val, y_pred))
print(r2_score(y_val, y_pred))

1191.5659819434493
1419829.4893248568
891.800142281378
0.509283640152049


In [160]:
# 모델2 | 랜덤포레스트, RandomforestRegressor
from sklearn.ensemble import RandomForestRegressor
rfr_model = RandomForestRegressor(random_state=2024)

rfr_model.fit(x_train, y_train)
y_pred = rfr_model.predict(x_val)

print(rmse(y_val, y_pred))
print(mean_squared_error(y_val, y_pred))
print(mean_absolute_error(y_val, y_pred))
print(r2_score(y_val, y_pred))

1115.2505641044197
1243783.8207352262
779.2212411979472
0.5701279107541344


In [165]:
# 모델3 | LightGBM (sklearn 아님 자체 라이브러리)
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(random_state=2024, verbose=-1)

lgb_model.fit(x_train, y_train)
y_pred = lgb_model.predict(x_val)

print(rmse(y_val, y_pred))
print(mean_squared_error(y_val, y_pred))
print(mean_absolute_error(y_val, y_pred))
print(r2_score(y_val, y_pred))

1117.95824022241
1249830.626881188
773.8531597106734
0.5680380353691235


In [166]:
# 예측 및 결과 파일 생성

In [ ]:
# LightGBM 모델 사용하기
pred = lgb_model.predict(test)
pred

array([1326.87699136,  908.47456812, 1778.03318278, ..., 4028.80564441,
        880.86733791, 1342.6291479 ])

In [173]:
# 파일 생성
# df 형태로 / 컬럼명은 pred / 파일명은 'result.csv' / index False로
result = pd.DataFrame({'pred': pred})
result.to_csv('result.csv', index=False)

In [174]:
# 불러와서 확인
pd.read_csv('result.csv')

,pred
0,1326.876991
1,908.474568
2,1778.033183
3,1451.008861
4,2410.334557
...,...
1700,300.637791
1701,529.104255
1702,4028.805644
1703,880.867338
